In [98]:
#import all the necessary libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import hvplot.pandas
import warnings
warnings.filterwarnings("ignore")
from scipy import stats 
import numpy as np
import requests
import json
from config import api_key
from scipy.constants import convert_temperature


In [71]:
 #designate the csv file data path
data_load = Path ('../Resources/wine_data.csv')

# uniform language
wine_df = pd.read_csv(data_load,encoding="ISO-8859-1")

# removing duplicates
wine_df = wine_df.drop(["Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12"], axis=1)
wine_df = wine_df.drop_duplicates()

# rennaming and capitalization of data
wine_df.rename(columns={'country':'Country'}, inplace=True)
wine_df.rename(columns={'description':'Description'}, inplace=True)
wine_df.rename(columns={'points':'Points'}, inplace=True)
wine_df.rename(columns={'price ($)':'Price ($)'}, inplace=True)
wine_df.rename(columns={'region':'Region'}, inplace=True)
wine_df.rename(columns={'title':'Title'}, inplace=True)
wine_df.rename(columns={'variety':'Variety'}, inplace=True)

#change types from objects to strings, anything that is a number to a float64/int
wine_df['Country']= wine_df['Country'].astype('string')
wine_df['Description']= wine_df['Description'].astype('string')
wine_df['Region']= wine_df['Region'].astype('string')
wine_df['Title']= wine_df['Title'].astype('string')
wine_df['Variety']= wine_df['Variety'].astype('string')
wine_df['Price ($)']=pd.to_numeric(wine_df['Price ($)'],errors='coerce')
wine_df['Points']=pd.to_numeric(wine_df['Points'],errors='coerce')

#drop unnecessary values
wine_df = wine_df.dropna(axis=0)

#print first five rows of data frame
wine_df.head()



,Country,Description,Points,Price ($),Region,Title,Variety
2,US,"Tart and snappy, the flavors of lime flesh and...",87.0,14.0,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",87.0,13.0,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling
4,US,"Much like the regular bottling from 2012, this...",87.0,65.0,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir
5,Spain,Blackberry and raspberry aromas show a typical...,87.0,15.0,Navarra,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot
6,Italy,"Here's a bright, informal red that opens with ...",87.0,16.0,Vittoria,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato


In [72]:
# df types
wine_df.dtypes

Country        string[python]
Description    string[python]
Points                float64
Price ($)             float64
Region         string[python]
Title          string[python]
Variety        string[python]
dtype: object

In [73]:
# df count
wine_df.count()

Country        93576
Description    93576
Points         93576
Price ($)      93576
Region         93576
Title          93576
Variety        93576
dtype: int64

In [74]:
#find the different columns that are in data frame
wine_df.columns 

Index(['Country', 'Description', 'Points', 'Price ($)', 'Region', 'Title',
       'Variety'],
      dtype='object')

In [75]:
#isolate the top 10 wines based on variety data
df= wine_df['Variety'].value_counts()
threshold = 2500
drop_df = df[df > threshold].index

clean_wine_df = wine_df[wine_df['Variety'].isin(drop_df)==True].reset_index(drop=True)

clean_wine_df

,Country,Description,Points,Price ($),Region,Title,Variety
0,US,"Much like the regular bottling from 2012, this...",87.0,65.0,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir
1,US,"Soft, supple plum envelopes an oaky structure ...",87.0,19.0,Napa Valley,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon
2,US,"Slightly reduced, this wine offers a chalky, t...",87.0,34.0,Alexander Valley,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon
3,US,Building on 150 years and six generations of w...,87.0,12.0,Central Coast,Mirassou 2012 Chardonnay (Central Coast),Chardonnay
4,US,Ripe aromas of dark berries mingle with ample ...,87.0,23.0,Virginia,Quiévremont 2012 Vin de Maison Red (Virginia),Red Blend
...,...,...,...,...,...,...,...
50916,Italy,"A blend of 65% Cabernet Sauvignon, 30% Merlot ...",90.0,20.0,Terre Siciliane,Feudo Principi di Butera 2012 Symposio Red (Te...,Red Blend
50917,US,There's no bones about the use of oak in this ...,90.0,35.0,Napa Valley,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay
50918,US,This opens with herbaceous dollops of thyme an...,90.0,35.0,Napa Valley,Hendry 2012 Blocks 7 & 22 Zinfandel (Napa Valley),Zinfandel
50919,US,This Zinfandel from the eastern section of Nap...,90.0,22.0,Chiles Valley,Houdini 2011 Zinfandel (Chiles Valley),Zinfandel


In [76]:
#check new df types 
clean_wine_df.dtypes

Country        string[python]
Description    string[python]
Points                float64
Price ($)             float64
Region         string[python]
Title          string[python]
Variety        string[python]
dtype: object

In [77]:
#make sure it printed out the top varieties
clean_wine_df['Variety'].value_counts()

Variety
Pinot Noir                  10841
Chardonnay                   9293
Cabernet Sauvignon           7738
Red Blend                    6940
Bordeaux-style Red Blend     4754
Syrah                        3345
Sauvignon Blanc              2930
Rosé                         2557
Zinfandel                    2523
Name: count, dtype: Int64

In [78]:
#isolate the wines that have points of 90 or avbove
high_points = clean_wine_df.loc[(clean_wine_df['Points']>= 90)]

#check that the correct points were isolated
high_points['Points'].value_counts()

Points
90.0     5899
91.0     4948
92.0     4378
93.0     3148
94.0     1933
95.0      771
96.0      273
97.0      119
98.0       41
99.0       15
100.0       9
Name: count, dtype: int64

In [79]:
#find the countries that produced the 90 or above wines
high_points['Country'].value_counts()

Country
US           15823
France        3522
Italy         1337
Australia      303
Spain          290
Argentina      212
Canada          47
Name: count, dtype: Int64

In [80]:
#create map for countries using their longitude and latitude
coordinates = {'longitude': [-95.7129,12.496366,2.213749, -4.00000000, 135.0000,-63.616672,-95.00],
                'latitude': [37.0902,41.902782, 46.227638, 40.00000000,-25.0000,-38.416097,60.00], 
               'Country': ['US', 'Italy', 'France', 'Spain', 'Australia', 'Argentina', 'Canada']}

coordinates_df= pd.DataFrame(coordinates)

coordinates_df

,longitude,latitude,Country
0,-95.712900,37.090200,US
1,12.496366,41.902782,Italy
2,2.213749,46.227638,France
3,-4.000000,40.000000,Spain
4,135.000000,-25.000000,Australia
5,-63.616672,-38.416097,Argentina
6,-95.000000,60.000000,Canada


In [81]:
#using the coordinates_df created above, map the points
country_map= coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM"
)

country_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]

In [85]:
#isolate the wines that are scored the highest (100)
highest_points_df = high_points.loc[(high_points['Points']==100)]

highest_points_df


,Country,Description,Points,Price ($),Region,Title,Variety
18682,US,Tasted in a flight of great and famous Napa wi...,100.0,200.0,Napa Valley,Cardinale 2006 Cabernet Sauvignon (Napa Valley),Cabernet Sauvignon
23700,France,"This is a magnificently solid wine, initially ...",100.0,150.0,Saint-Julien,Château Léoville Barton 2010 Saint-Julien,Bordeaux-style Red Blend
35721,France,This new release from a great vintage for Char...,100.0,617.0,Champagne,Salon 2006 Le Mesnil Blanc de Blancs Brut Char...,Chardonnay
44204,France,"Almost black in color, this stunning wine is g...",100.0,1500.0,Pauillac,Château Lafite Rothschild 2010 Pauillac,Bordeaux-style Red Blend
44205,France,This is the finest Cheval Blanc for many years...,100.0,1500.0,Saint-Émilion,Château Cheval Blanc 2010 Saint-Émilion,Bordeaux-style Red Blend
44206,France,"A hugely powerful wine, full of dark, brooding...",100.0,359.0,Saint-Julien,Château Léoville Las Cases 2010 Saint-Julien,Bordeaux-style Red Blend
44948,US,In 2005 Charles Smith introduced three high-en...,100.0,80.0,Columbia Valley (WA),Charles Smith 2006 Royal City Syrah (Columbia ...,Syrah
46462,US,This wine dazzles with perfection. Sourced fro...,100.0,450.0,Sonoma County,Verité 2007 La Muse Red (Sonoma County),Bordeaux-style Red Blend
48502,US,Initially a rather subdued Frog; as if it has ...,100.0,80.0,Walla Walla Valley (WA),Cayuse 2008 Bionic Frog Syrah (Walla Walla Val...,Syrah


In [86]:
#create df for highest rated wines
high_region_coordinates = {'Region': ['Napa Valley', 'Saint-Julien', 'Champagne', 'Pauillac',
                                       'Saint-Emilion', 'Columbia Valley (WA)',
                                       'Sonoma County', 'Walla Walla Valley (WA)'],
                                       'latitude': [38.297539, 45.15719931429095, 48.957500,45.19644,44.88333000, 49.0000,38.52529000,46.064583],
                                    'longitude': [ -122.286865,  -0.7391816604296135, 4.365000,-0.74873,-0.15000000, -122.0833,-122.92254000, -118.343018]}

high_region_coordinates_df= pd.DataFrame(high_region_coordinates)

high_region_coordinates_df

,Region,latitude,longitude
0,Napa Valley,38.297539,-122.286865
1,Saint-Julien,45.157199,-0.739182
2,Champagne,48.957500,4.365000
3,Pauillac,45.196440,-0.748730
4,Saint-Emilion,44.883330,-0.150000
5,Columbia Valley (WA),49.000000,-122.083300
6,Sonoma County,38.525290,-122.922540
7,Walla Walla Valley (WA),46.064583,-118.343018


In [87]:
#using the high_reion_coordinates_df plot the points on a map
high_region_map= high_region_coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM"
)

high_region_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]

In [90]:
#create a new df that will hold weather data for the regions

weather_data = high_region_coordinates_df

weather_data_df = pd.DataFrame(weather_data)

weather_data_df

,Region,latitude,longitude
0,Napa Valley,38.297539,-122.286865
1,Saint-Julien,45.157199,-0.739182
2,Champagne,48.957500,4.365000
3,Pauillac,45.196440,-0.748730
4,Saint-Emilion,44.883330,-0.150000
5,Columbia Valley (WA),49.000000,-122.083300
6,Sonoma County,38.525290,-122.922540
7,Walla Walla Valley (WA),46.064583,-118.343018


In [92]:
#create a for loop to go through each lat and lon and pull temp/precipitation from API
for index, row in weather_data_df.iterrows():
    #define lat and lon variable by assigning it to its respective row from weather_data_df
    lat = row["latitude"]
    lon = row["longitude"]
    #create the base url that will be used for the API
    url = 'https://history.openweathermap.org/data/2.5/aggregated/year?'
    #create the query_url that has the parameters needed to access and find the data
    query_url = f"{url}lat={lat}&lon={lon}&appid={api_key}"
    #create the response by getting the query_url in json format
    response = requests.get(query_url).json()
    #as it loops through the lat/lon, have it append the temp/precipitation to its respective columns
    weather_data_df.loc[index, "Average Temperature (kelvin)"] = response["result"][1]["temp"]['mean']
    weather_data_df.loc[index, "Max Precipitation (mm)"] = response["result"][1]["precipitation"]['max']
#at the end of the for loop print the df to make sure information was added correctly 
weather_data_df

,Region,latitude,longitude,Average Temperature (kelvin),Max Precipitation (mm)
0,Napa Valley,38.297539,-122.286865,281.37,3.0
1,Saint-Julien,45.157199,-0.739182,281.51,0.9
2,Champagne,48.957500,4.365000,279.50,3.0
3,Pauillac,45.196440,-0.748730,281.51,0.9
4,Saint-Emilion,44.883330,-0.150000,281.51,0.9
5,Columbia Valley (WA),49.000000,-122.083300,275.26,0.9
6,Sonoma County,38.525290,-122.922540,280.44,3.0
7,Walla Walla Valley (WA),46.064583,-118.343018,268.94,0.3


In [102]:
weather_data_df.dtypes

Region                           object
latitude                        float64
longitude                       float64
Average Temperature (kelvin)    float64
Max Precipitation (mm)          float64
dtype: object

In [111]:
temp1= np.array([281.37])
temp2 = np.array([281.51])
temp3= np.array([279.50])
temp4= np.array([281.51])
temp5= np.array([275.26])
temp6= np.array([280.44])
temp7= np.array([268.94])
                              

In [129]:
temp1_fah= convert_temperature(np.array([temp1]), 'Kelvin', 'Fahrenheit')
temp2_fah = convert_temperature(np.array([temp2]), 'Kelvin', 'Fahrenheit')
temp3_fah = convert_temperature(np.array([temp3]), 'Kelvin', 'Fahrenheit')
temp4_fah = convert_temperature(np.array([temp4]), 'Kelvin', 'Fahrenheit')
temp5_fah = convert_temperature(np.array([temp5]), 'Kelvin', 'Fahrenheit')
temp6_fah = convert_temperature(np.array([temp6]), 'Kelvin', 'Fahrenheit')
temp7_fah = convert_temperature(np.array([temp7]), 'Kelvin', 'Fahrenheit')


dtype('float64')

In [114]:
print(temp1_fah)
print(temp2_fah)
print(temp3_fah)
print(temp4_fah)
print(temp5_fah)
print(temp6_fah)
print(temp7_fah)

[[46.796]]
[[47.048]]
[[43.43]]
[[47.048]]
[[35.798]]
[[45.122]]
[[24.422]]


In [128]:

integer_array1 = temp1_fah.astype(int)
integer_array2 = temp2_fah.astype(int)
integer_array3 = temp3_fah.astype(int)
integer_array4 = temp4_fah.astype(int)
integer_array5 = temp5_fah.astype(int)
integer_array6 = temp6_fah.astype(int)
integer_array7 = temp7_fah.astype(int)


dtype('int32')

In [126]:
temp_df = pd.DataFrame ({'Avg. Temp (Fahreinheit)': [integer_array1, integer_array2,integer_array3,integer_array4,
                                                     integer_array5,integer_array6,integer_array7]})

temp_df


,Avg. Temp (Fahreinheit)
0,[[46]]
1,[[47]]
2,[[43]]
3,[[47]]
4,[[35]]
5,[[45]]
6,[[24]]


In [127]:
def remove_extra_brackets(value):
    return value.strip('[[').strip(']]')

temp_df = temp_df.applymap(remove_extra_brackets)

temp_df


AttributeError: 'numpy.ndarray' object has no attribute 'strip'